In [1]:
from functions import *
from torch.nn.utils import clip_grad_value_
from torch.optim import AdamW,Adam,SGD
import math
LOCAL = True
if LOCAL:
    dataPath = '/home/zhenlan/Desktop/Projects/HMS/Data/'

In [2]:
# HP
config = {
    "d_model": 512,
    "n_layer": 12,
    "rms_norm": False,
    "normOverChannel": True,
    "in_channels": 20, # if EEG
    "seqLen": 256,
    "dataFolder": "train_eegs_LocalNorm",
    # "classWeight": 0.0
}

config.update(fixed_config)
config = Config(config)

# other training parameters
epochs = 10
batch_size = 64
accumulation_steps = 2
lr = 6e-5
clip = 6e-3

Data

In [3]:
train = pd.read_pickle(dataPath+'train_data.pkl')
val = pd.read_pickle(dataPath+'val_data.pkl')
# trainData = eegData(train,dataPath,config.dataFolder,seqLen=config.seqLen)
# valData = eegData(val,dataPath,config.dataFolder,seqLen=config.seqLen)
trainData = eegDataXonly(train,dataPath,config.dataFolder,seqLen=config.seqLen)
valData = eegDataXonly(val,dataPath,config.dataFolder,seqLen=config.seqLen)

train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [3]:
model = seq2seqModel(config).to('cuda')
trainable_params = list(model.parameters())
optimizer = AdamW(trainable_params,lr = lr,amsgrad=True)
# optimizer = Adam(trainable_params,lr = lr)
# optimizer = SGD(trainable_params,lr = lr)

In [5]:
# for i, (input_ids,targets) in enumerate(train_loader):
#         input_ids,targets = input_ids.to('cuda'),targets.to('cuda')
#         autoL,classL = model(input_ids,targets)
#         loss = autoL + config.classWeight * classL
#         loss.backward()
#         break

# for param in model.parameters():
#     if torch.any(torch.isnan(param.grad)):
#         print('NaN')

In [6]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, input_ids in enumerate(train_loader):
        input_ids = input_ids.to('cuda')
        loss = model(input_ids)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, input_ids in enumerate(val_loader):
                input_ids = input_ids.to('cuda')
                loss = model(input_ids)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 0.5643352270126343, eval loss 0.5241860747337341
epoch 1: train loss 0.5055997371673584, eval loss 0.5055713057518005
epoch 2: train loss 0.49228569865226746, eval loss 0.48613426089286804
epoch 3: train loss 0.4745284616947174, eval loss 0.45736682415008545
epoch 4: train loss 0.465613454580307, eval loss 0.46564626693725586
epoch 5: train loss 0.45951899886131287, eval loss 0.45764753222465515
epoch 6: train loss 0.4544704556465149, eval loss 0.45245009660720825
epoch 7: train loss 0.4458999037742615, eval loss 0.4487306773662567
epoch 8: train loss 0.4407556354999542, eval loss 0.4432801306247711
epoch 9: train loss 0.4364316463470459, eval loss 0.4364400804042816


In [8]:
# adjust seqLen
trainData.seqLen = int(trainData.seqLen * 2)
valData.seqLen = int(trainData.seqLen * 2)
batch_size = int(batch_size / 2)
# accumulation_steps = int(accumulation_steps * 2)
train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [11]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, input_ids in enumerate(train_loader):
        input_ids = input_ids.to('cuda')
        loss = model(input_ids)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, input_ids in enumerate(val_loader):
                input_ids = input_ids.to('cuda')
                loss = model(input_ids)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 0.423208087682724, eval loss 0.4182685315608978
epoch 1: train loss 0.4170055389404297, eval loss 0.4170435965061188
epoch 2: train loss 0.41220834851264954, eval loss 0.40758243203163147
epoch 3: train loss 0.4099055528640747, eval loss 0.4065611660480499
epoch 4: train loss 0.4033583998680115, eval loss 0.4019213914871216
epoch 5: train loss 0.4022318720817566, eval loss 0.3959902822971344
epoch 6: train loss 0.39832955598831177, eval loss 0.4028504490852356
epoch 7: train loss 0.39300668239593506, eval loss 0.3979494869709015
epoch 8: train loss 0.39111995697021484, eval loss 0.39449262619018555
epoch 9: train loss 0.3881307542324066, eval loss 0.3912297487258911


In [18]:
# adjust seqLen
trainData.seqLen = int(trainData.seqLen * 2)
valData.seqLen = int(trainData.seqLen * 2)
batch_size = int(batch_size / 2)
# accumulation_steps = int(accumulation_steps * 2)
train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [23]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, input_ids in enumerate(train_loader):
        input_ids = input_ids.to('cuda')
        loss = model(input_ids)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, input_ids in enumerate(val_loader):
                input_ids = input_ids.to('cuda')
                loss = model(input_ids)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 0.3890700340270996, eval loss 0.39127740263938904
epoch 1: train loss 0.37898337841033936, eval loss 0.37718817591667175
epoch 2: train loss 0.3727087080478668, eval loss 0.3745807707309723
epoch 3: train loss 0.3680777847766876, eval loss 0.3733740448951721
epoch 4: train loss 0.3635190427303314, eval loss 0.3750753104686737
epoch 5: train loss 0.35890939831733704, eval loss 0.3664422929286957
epoch 6: train loss 0.3572143018245697, eval loss 0.36590519547462463
epoch 7: train loss 0.35246413946151733, eval loss 0.3540668487548828
epoch 8: train loss 0.3504982888698578, eval loss 0.3626132607460022
epoch 9: train loss 0.347641259431839, eval loss 0.35376644134521484


In [24]:
# adjust seqLen
trainData.seqLen = int(trainData.seqLen * 2)
valData.seqLen = int(trainData.seqLen * 2)
batch_size = int(batch_size / 2)
# accumulation_steps = int(accumulation_steps * 2)
train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [25]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, input_ids in enumerate(train_loader):
        input_ids = input_ids.to('cuda')
        loss = model(input_ids)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, input_ids in enumerate(val_loader):
                input_ids = input_ids.to('cuda')
                loss = model(input_ids)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 0.3441787362098694, eval loss 0.3506990671157837
epoch 1: train loss 0.33828991651535034, eval loss 0.35045817494392395
epoch 2: train loss 0.33368730545043945, eval loss 0.34320536255836487
epoch 3: train loss 0.328596830368042, eval loss 0.3480270802974701
epoch 4: train loss 0.32515719532966614, eval loss 0.33498314023017883
epoch 5: train loss 0.3217135965824127, eval loss 0.3361016511917114
epoch 6: train loss 0.31889957189559937, eval loss 0.33484527468681335
epoch 7: train loss 0.3152419626712799, eval loss 0.3323552906513214
epoch 8: train loss 0.3136831820011139, eval loss 0.32940948009490967
epoch 9: train loss 0.3090258836746216, eval loss 0.3315391540527344


In [7]:
# save model
model_name = 'model1'
torch.save(model.state_dict(), '/home/zhenlan/Desktop/Projects/HMS/Model_state/'+model_name+'.pth')
import pickle
with open('/home/zhenlan/Desktop/Projects/HMS/Model_state/'+model_name+'.pkl', 'wb') as file:
    pickle.dump(config, file)
# model.load_state_dict(torch.load(PATH))

In [4]:
train = pd.read_pickle(dataPath+'train_data.pkl')
val = pd.read_pickle(dataPath+'val_data.pkl')
trainData = eegDataXonly(train,dataPath,config.dataFolder,seqLen=10000)
valData = eegDataXonly(val,dataPath,config.dataFolder,seqLen=10000)
batch_size = 1
accumulation_steps = 4
train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [5]:
# load model
model_name = 'model1'
model.load_state_dict(torch.load('/home/zhenlan/Desktop/Projects/HMS/Model_state/'+model_name+'.pth'))

<All keys matched successfully>

In [30]:
# # adjust seqLen
# trainData.seqLen = 10000
# valData.seqLen = 10000
# batch_size = 1
# accumulation_steps = 4
# train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
# val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [6]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, input_ids in enumerate(train_loader):
        input_ids = input_ids.to('cuda')
        loss = model(input_ids)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, input_ids in enumerate(val_loader):
                input_ids = input_ids.to('cuda')
                loss = model(input_ids)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 0.3123624324798584, eval loss 0.328789085149765
epoch 1: train loss 0.29967764019966125, eval loss 0.3213753402233124
epoch 2: train loss 0.2900429368019104, eval loss 0.3142571449279785
epoch 3: train loss 0.28209131956100464, eval loss 0.31462356448173523
epoch 4: train loss 0.27488476037979126, eval loss 0.3100687563419342
epoch 5: train loss 0.26925671100616455, eval loss 0.30670639872550964
epoch 6: train loss 0.2631770074367523, eval loss 0.3039543330669403
epoch 7: train loss 0.258156955242157, eval loss 0.3038141131401062
epoch 8: train loss 0.25356540083885193, eval loss 0.3010229170322418
epoch 9: train loss 0.2493651658296585, eval loss 0.3004760146141052


Fine tune

In [4]:
# Data
batch_size = 2
accumulation_steps = 4
lr = 6e-6
clip = 6e-4
train = pd.read_pickle(dataPath+'train_data.pkl')
val = pd.read_pickle(dataPath+'val_data.pkl')
trainData = eegData(train,dataPath,config.dataFolder,seqLen=10000)
valData = eegData(val,dataPath,config.dataFolder,seqLen=10000)

train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [11]:
optimizer = AdamW(trainable_params,lr = lr,amsgrad=True)

In [12]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, (input_ids,target) in enumerate(train_loader):
        input_ids,target = input_ids.to('cuda'), target.to('cuda')
        loss = model(input_ids,target)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, (input_ids,target) in enumerate(val_loader):
                input_ids,target = input_ids.to('cuda'), target.to('cuda')
                loss = model(input_ids,target)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 1.1511163711547852, eval loss 0.9513724446296692
epoch 1: train loss 0.8458009958267212, eval loss 0.8123239874839783
epoch 2: train loss 0.7216739058494568, eval loss 0.762212336063385
epoch 3: train loss 0.6343607306480408, eval loss 0.7389225959777832
epoch 4: train loss 0.5664060711860657, eval loss 0.7422981858253479
epoch 5: train loss 0.511198878288269, eval loss 0.7324237823486328
epoch 6: train loss 0.4618319571018219, eval loss 0.7608739733695984
epoch 7: train loss 0.41717249155044556, eval loss 0.762435257434845
epoch 8: train loss 0.36739709973335266, eval loss 0.8094151020050049


KeyboardInterrupt: 

Train from start

In [3]:
# Data
batch_size = 2
accumulation_steps = 4
lr = 6e-6
clip = 6e-4
train = pd.read_pickle(dataPath+'train_data.pkl')
val = pd.read_pickle(dataPath+'val_data.pkl')
trainData = eegData(train,dataPath,config.dataFolder,seqLen=10000)
valData = eegData(val,dataPath,config.dataFolder,seqLen=10000)

train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [4]:
model = seq2seqModel(config).to('cuda')
trainable_params = list(model.parameters())
optimizer = AdamW(trainable_params,lr = lr,amsgrad=True)

In [5]:
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # train
    for i, (input_ids,target) in enumerate(train_loader):
        input_ids,target = input_ids.to('cuda'), target.to('cuda')
        loss = model(input_ids,target)
        loss.backward()
        train_loss += loss.detach().cpu()
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
            # print(i,train_loss)
    train_loss /= (i+1)

    # eval
    eval_loss = 0
    model.eval()
    with torch.no_grad():
        for i, (input_ids,target) in enumerate(val_loader):
                input_ids,target = input_ids.to('cuda'), target.to('cuda')
                loss = model(input_ids,target)
                eval_loss += loss.detach().cpu()
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train loss {train_loss}, eval loss {eval_loss}")
    model.train()

epoch 0: train loss 1.308092713356018, eval loss 1.23896062374115
epoch 1: train loss 1.2475287914276123, eval loss 1.2214336395263672
epoch 2: train loss 1.2331287860870361, eval loss 1.209027886390686
epoch 3: train loss 1.2163844108581543, eval loss 1.1937953233718872
epoch 4: train loss 1.1912814378738403, eval loss 1.1904999017715454
epoch 5: train loss 1.1722325086593628, eval loss 1.1823939085006714


KeyboardInterrupt: 